Use this utility to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [6]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['Headquarters Location', 'Date added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,Industrials,Industrial Conglomerates
AOS,A. O. Smith,Industrials,Building Products
ABT,Abbott,Health Care,Health Care Equipment
ABBV,AbbVie,Health Care,Biotechnology
ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,Health Care,Health Care Equipment


In [7]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [8]:
# Map sp500 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp500.index = sp500.index.str.replace('.', '-', regex=False)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,US Stocks:Industrials,Industrial Conglomerates
AOS,A. O. Smith,US Stocks:Industrials,Building Products
ABT,Abbott,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie,US Stocks:Healthcare,Biotechnology
ACN,Accenture,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,US Stocks:Healthcare,Health Care Equipment


In [9]:
# Make symbols list.
symbols = list(sp500.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BK BBWI BAX BDX BRK-B BBY BIO TECH BIIB BLK BX BA BKNG BWA BXP BSX BMY AVGO BR BRO BF-B BLDR BG CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE COR CNC CNP CDAY CF CHRW CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FLT FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GEHC GEN GNRC GD GE GIS GM GPC GILD GPN GL GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUBB

In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2023-12-22,488.30,354.93,598.75,697.55,139.60,61.84,193.60,374.58,230.12,529.05,153.42,122.17,412.97,846.72,42.98,248.03,155.71,275.82,266.34,562.28
2023-12-26,492.79,357.34,598.26,701.23,143.41,62.56,193.05,374.66,231.16,529.70,153.41,123.55,416.00,849.53,43.27,251.99,156.83,274.96,266.22,561.27
2023-12-27,494.17,355.88,596.08,703.76,146.07,62.68,193.15,374.07,232.62,531.65,153.34,123.73,413.60,874.37,43.64,250.94,157.94,274.64,266.72,565.62
2023-12-28,495.22,354.93,595.52,702.46,148.76,63.08,193.58,375.28,233.13,532.94,153.38,124.16,411.06,881.70,43.77,253.58,158.95,273.24,265.58,570.60
2023-12-29,495.22,352.12,596.60,706.49,147.41,61.41,192.53,376.04,231.34,530.79,151.94,124.09,405.33,878.29,43.67,252.97,160.33,272.37,263.14,565.65


In [12]:
# Sample symbol.
symbol = 'MMM'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

-1.99598963573151

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.21916621882028162

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.10241509703980027

In [17]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [18]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2023-08-31   -0.03
2023-09-30   -0.12
2023-10-31   -0.03
2023-11-30    0.11
2023-12-31    0.10
Freq: M, Name: MMM, Length: 108, dtype: float64

In [19]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.25389051254866307

In [20]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [21]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [22]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')